In [2]:
import os
from openai import OpenAI
import re
import json
from IPython.display import clear_output

In [4]:
key = 'key'
os.environ['OPENAI_API_KEY'] = key

In [6]:
def ai_get_recipe_json(client, dish, experience, servings, allergies):
    """Fetches a recipe by name and experience level using AI and returns JSON."""
    recipe = client.chat.completions.create(
      model="gpt-4o",
      response_format={"type": "json_object"},
      messages=[
        {"role": "system", "content": "You are a chef, skilled at creating and finding recipes for anyone and anything designed to output JSON."},
        {"role": "user", "content": f"Create a recipe for {dish} for a person of {experience} experience. Include amounts of each ingredient for {servings} and avoid {allergies} due to allergies. Be sure to inlcude the name of the dish and the total time."}
      ]
    ).choices[0].message.content
    return recipe

def convert_to_json(client, recipe):
    """Converts existing recipe to JSON using AI."""
    json_recipe = client.chat.completions.create(
      model="gpt-4o",
      response_format={"type": "json_object"},
      messages=[
        {"role": "system", "content": "You are a recipe reader, skilled at reading recipes and converting them to JSON."},
        {"role": "user", "content": recipe},
        {"role": "user", "content": "Convert that recipe to JSON. Include recipe name, total time, servings, and amounts for each ingredient"}
      ]
    ).choices[0].message.content
    return json_recipe

def adjust_servings(client, recipe, servings):
    """Modifies the recipe for the new servings."""
    # adjust recipe
    new_recipe = client.chat.completions.create(
      model="gpt-4o",
      response_format={"type": "json_object"},
      messages=[
        {"role": "system", "content": "You are an recipe reader, skilled at adjusting ingredient amounts for a different number of servings, and designed to output in JSON"},
        {"role": "assistant", "content": recipe},
        {"role": "user", "content": f"I need to make that exact recipe, but for {servings} servings. Modify the servings listed and the ingredient amounts accordingly."}
      ]
    ).choices[0].message.content
    return new_recipe

def create_grocery_list_ai(client, ingredients, have):
    """Creates a grocery list for ingredients that user does not have."""
    # load ingredient list
    ingredients = json.loads(client.chat.completions.create(
      model="gpt-4o",
      response_format={"type": "json_object"},
      messages=[
        {"role": "system", "content": "You are an ingredient list reader, skilled at reading ingredient lists and converting them to JSON with ingredient-amount pairs."},
        {"role": "assistant", "content": ingredients},
        {"role": "user", "content": "Convert that ingredient list to JSON with ingredient-value pairs"}
      ]
    ).choices[0].message.content)

    # check which ingredients user has
    yes = ('y','Y','yes','Yes','YES')
    no = ('n','N','no','No','NO')
    for i in ingredients:
        if i in have:
            continue
        while True:
            check = input(f"Do you have {ingredients[i]} {i} (y/n)? ")
            if check in yes:
                have[i] = ingredients[i]
                break
            elif check in no:
                break
            print("Please enter 'y' or 'n'")

    # generate grocery list
    print("Generating grocery list...")
    list = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "system", "content": "You are a grocery list reader, skilled at reading multiple lists and synthesizing them to make a new one."},
        {"role": "assistant", "content": str(ingredients)},
        {"role": "user", "content": f"I have these ingredients: {str(have)}. I need these ingredients: {str(ingredients)}. Make me a grocery list of what I need but don't have, including the amount I should buy of each (1 14 oz can, 2 1lb packages, etc.)."}
      ]
    ).choices[0].message.content
    return list, have

def modify_recipe_ai(client, ingredients, recipe, have, missing):
    """Modifies the recipe to exclude missing ingredients using AI."""
    # load ingredient list
    ingredients = json.loads(client.chat.completions.create(
      model="gpt-4o",
      response_format={"type": "json_object"},
      messages=[
        {"role": "system", "content": "You are an ingredient list reader, skilled at reading ingredient lists and converting them to JSON with ingredient-amount pairs."},
        {"role": "assistant", "content": ingredients},
        {"role": "user", "content": "Convert that ingredient list to JSON with ingredient-value pairs"}
      ]
    ).choices[0].message.content)

    # check which ingredients user is missing
    yes = ('y','Y','yes','Yes','YES')
    no = ('n','N','no','No','NO')
    for i in ingredients:
        if i in have or i in missing:
            continue
        while True:
            check = input(f"Do you have {ingredients[i]} {i} (y/n)? ")
            if check in yes:
                have[i] = ingredients[i]
                break
            elif check in no:
                missing[i] = ingredients[i]
                break
            print("Please enter 'y' or 'n'")

    # modify recipe
    print("Generating modified recipe..", end="")
    new_recipe = client.chat.completions.create(
      model="gpt-4o",
      response_format={"type": "json_object"},
      messages=[
        {"role": "system", "content": "You are an accomodating chef, skilled at modifying recipes so they don't require certain ingredients, and designed to output the modified recipe in JSON."},
        {"role": "assistant", "content": recipe},
        {"role": "user", "content": f"I like this recipe: {recipe} but I don't have these ingredients: {str(missing)}. Modify the recipe so it is similar but avoids or replaces the ingredients I don't have."}
      ]
    ).choices[0].message.content
    return new_recipe, have, missing

def new_recipe_ai(client, dish, experience, servings, allergies, recipe, missing):
    """Fetches a new recipe based on dislikes of old recipe by name and experience level using AI and returns JSON."""
    dislikes = input("What do you dislike about the current recipe? ")
    print("Generating new recipe..", end="")
    new_recipe = client.chat.completions.create(
      model="gpt-4o",
      response_format={"type": "json_object"},
      messages=[
        {"role": "system", "content": "You are a chef, skilled at creating and finding recipes for anyone and anything designed to output JSON."},
        {"role": "assistant", "content": recipe},
        {"role": "user", "content": f"Create a recipe for {dish} for a person of {experience} experience, knowing that these were their dislikes of the previous recipe: {dislikes}. Include amounts of each ingredient for {servings} and avoid {allergies} due to allergies. Be sure to inlcude the name of the dish and the total time."}
      ]
    ).choices[0].message.content
    return new_recipe

def print_recipe_ai(client, recipe, servings):
    """Prints the full recipe using AI."""
    # get name of recipe
    print(".", end="")
    name = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "system", "content": "You are a JSON reader, skilled at reading JSON and pulling out pieces you are asked for, whether the keys have the exact same name or not."},
        {"role": "assistant", "content": recipe},
        {"role": "user", "content": "Give me only the name of that recipe. Remove all quotation marks and replace underscores with a space."}
      ]
    ).choices[0].message.content

    # get cook time of recipe
    print(".", end="")
    time = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "system", "content": "You are a JSON reader, skilled at reading JSON and pulling out pieces you are asked for, whether the keys have the exact same name or not."},
        {"role": "assistant", "content": recipe},
        {"role": "user", "content": "Give me only the time required of that recipe, in the form '{number} minutes'. Remove all quotation marks and replace underscores with a space."}
      ]
    ).choices[0].message.content

    # get ingredient list
    print(".", end="")
    ingredients = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "system", "content": "You are a JSON reader, skilled at reading JSON and pulling out pieces you are asked for, whether the keys have the exact same name or not."},
        {"role": "assistant", "content": recipe},
        {"role": "user", "content": "Give me only the ingredients/amounts of that recipe, with each ingredient on its own line, bulleted in the format 'ingredient, amount'. Remove all quotation marks and replace underscores with a space."}
      ]
    ).choices[0].message.content

    # get directions
    print(".", end="")
    steps = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "system", "content": "You are a JSON reader, skilled at reading JSON and pulling out pieces you are asked for, whether the keys have the exact same name or not."},
        {"role": "assistant", "content": recipe},
        {"role": "user", "content": "Give me only the steps of that recipe, numbered, with each step on its own line. Remove all quotation marks and replace underscores with a space."}
      ]
    ).choices[0].message.content

    # format and print parts as full recipe
    print("\nHere's a suggested recipe based on your input:")
    print(f"\n{name}\nTotal time: {time}\nServings: {servings}\n\nIngredients:\n{ingredients}\n\nDirections:\n{steps}")
    return name, time, ingredients, steps

def to_file(name, time, servings, ingredients, steps):
    """Prints recipe to file"""
    # save printed recipe text
    text = f"\n{name}\nTotal time: {time}\nServings: {servings}\n\nIngredients:\n{ingredients}\n\nDirections:\n{steps}"
    while True:
        file_name = input("Enter file name to save the file to (must be a .txt; if it does not exist, the file will be created; if it does, it will be overwritten; make sure the file is not open): ")
        try: # ensure file can open
            f = open(file_name, "w")
        except:
            print("An error ocurred when opening this file. Please try again.")
            continue
        else:
            pass
            
        try: # ensure file can write
            f.write(text)
        except:
            print("An error ocurred when writing to this file. Please try again.")
        else:
            break
    return file_name

def clear():
    """Displays the first few lines after clearing output"""
    clear_output()
    print("Welcome to the AI-powered Cooking Assistant!\n")

def walkthrough_recipe(name, ingredients, steps):
    """Walks the user through the recipe step by step."""
    print(f"Let's start cooking {name}!")
    print("First, get your ingredients together:")
    print(ingredients)
    steps = steps.split("\n")
    input("Press enter once you have all of your ingredients together.")
    print("\nNow let's cook!\nPress enter after completing each step to move on to the next.")
    for step in steps:
        print("\n")
        input(step)
    print("\nRecipe complete! Enjoy your meal!\n")

In [8]:
# Main function to run the cooking assistant
def cooking_assistant():
    client = OpenAI()
    missing = {}
    have = {}
    
    print("Welcome to the AI-powered Cooking Assistant!")

    while True:
        # User options
        print("\nWhat would you like to do?")
        print("1. Create a recipe.")
        print("2. Upload and modify my own recipe")
        
        choice = input("Enter your choice (1/2): ").strip()
        
        if choice == "1": # create a recipe
            print("\nLet's create a recipe.")
            # Ask for user details
            experience_level = input("What's your cooking experience level? ")
            allergies = input("Do you have any allergies? (comma separated): ")
            servings = int(input("How many servings are you cooking for? (number): ").strip())
            
            # Ask for desired dish
            dish_name = input("What do you want to cook? (e.g., chocolate cupcakes, pasta, pastries, dinner): ")
            print("\nGenerating recipe..", end="")
            recipe = ai_get_recipe_json(client, dish_name, experience_level, servings, allergies)
            break
    
        elif choice == "2": # upload and modify a recipe
            print("\nLet's modify your recipe.")
            file_name = input("Ensure your recipe is in a .txt file. Enter the file path here: ")
            while True:
                try: # ensure file can open
                    f = open(file_name, "r")
                except:
                    print("An error ocurred when opening this file. Please try again.")
                    continue
                else:
                    pass
                    
                try: # ensure file can write
                    recipe = f.read()
                except:
                    print("An error ocurred when reading this file. Please try again.")
                else:
                    break
            print("File read successfully.")
            servings = input("How many servings are you cooking for (same or different from what the original recipe makes)? ")
            print("Adjusting recipe..", end="")
            recipe = convert_to_json(client, recipe)
            recipe = adjust_servings(client, recipe, servings)
            dish_name = None
            break
    
        else: # something other than 1 or 2 was entered - tell user and ask again
            print("Invalid choice. Please enter only the number.")

    # Print recipe
    name, time, ingredients, steps = print_recipe_ai(client, recipe, servings)
    if dish_name is None:
        dish_name = name

    # Modify recipe until user likes it
    while True:
        # User options
        print("\nWhat would you like to do next?")
        print("1. I have all the ingredients - Start cooking!")
        print("2. Print this recipe.")
        print("3. I need to buy some ingredients - Create a grocery list.")
        print("4. I don't have some ingredients and can't buy them - Modify the recipe.")
        print("5. I don't like this recipe - Let's modify or make a new one.")
    
        choice = input("Enter your choice (1/2/3/4/5): ").strip()
    
        if choice == "1": # start cooking
            break

        elif choice == "2": # print the recipe
            print("\nLet's print your recipe.")
            file_name = to_file(name, time, servings, ingredients, steps)
            print(f"Your recipe has been saved to {file_name}")
    
        elif choice == "3": # create a grocery list
            print("\nLet's create your grocery list.")
            list, have = create_grocery_list_ai(client, ingredients, have)
            clear()
            print(list)
            input("Press enter once you have purchased all of your ingredients")
    
        elif choice == "4": # modify recipe
            print("\nLet's modify the recipe.")
            recipe, have, missing = modify_recipe_ai(client, ingredients, recipe, have, missing)
            clear()
            print("Generating modified recipe..", end="")
            name, time, ingredients, steps = print_recipe_ai(client, recipe, servings)
            input("Press enter once you have reviewed the new recipe. If you are still missing ingredients, you can modify again.")
    
        elif choice == "5": # new recipe
            print("\nLet's find another recipe.")
            recipe = new_recipe_ai(client, dish_name, experience_level, servings, allergies, recipe, missing)
            clear()
            print("Generating new recipe..", end="")
            name, time, ingredients, steps = print_recipe_ai(client, recipe, servings)
            input("Press enter once you have reviewed the new recipe. If you are still want to make changes, you can modify again.")
    
        else: # something other than 1, 2, 3, 4, or 5 was entered - tell user and ask again
            print("Invalid choice. Please enter only the number.")

    clear()
    walkthrough_recipe(name, ingredients, steps)

    # print recipe if necessary
    yes = ('y','Y','yes','Yes','YES')
    no = ('n','N','no','No','NO')
    while True:
        answer = input("In case you didn't print your recipe before, would you like to now (y/n)? ")
        if answer in yes:
            file_name = to_file(name, time, servings, ingredients, steps)
            print(f"Your recipe has been saved to {file_name}")
            break
        elif answer in no:
            break

    # sign off
    print("\nNow that you're done cooking, I'm done for now! Come back next time you need help from the AI-powered Cooking Assistant!")

In [14]:
#### Run the cooking assistant
if __name__ == "__main__":
    cooking_assistant()

Welcome to the AI-powered Cooking Assistant!

Let's start cooking Chocolate and Caramel Croquembouche!
First, get your ingredients together:
- Water, 250 ml
- Unsalted Butter, 100 g
- Salt, 1/2 tsp
- Sugar, 1 tsp
- All-purpose Flour, 150 g
- Large Eggs, 4
- Whole Milk, 500 ml
- Vanilla Extract, 1 tsp
- Egg Yolks, 6
- Granulated Sugar, 100 g
- Cornstarch, 50 g
- Unsalted Butter, 50 g
- Cocoa Powder, 50 g
- Unsalted Butter, 50 g
- Milk, 200 ml
- Powdered Sugar, 50 g
- Granulated Sugar, 300 g
- Water, 100 ml


Press enter once you have all of your ingredients together. 



Now let's cook!
Press enter after completing each step to move on to the next.




1. Preheat the oven to 200°C (400°F). Line two baking trays with parchment paper. 


2. In a medium saucepan, combine the water, butter, salt, and sugar. Bring to a boil. 


...

20. Dip each filled choux bun into the chocolate ganache substitute and then stack them into a cone shape on a serving platter, using caramel as glue. 


21. Drizzle additional caramel over the entire croquembouche for a decorative effect. 



Recipe complete! Enjoy your meal!



In case you didn't print your recipe before, would you like to now (y/n)?  n



Now that you're done cooking, I'm done for now! Come back next time you need help from the AI-powered Cooking Assistant!
